In [1]:
import torch

In [2]:
N = 10
D_in = 1
D_out = 1

torch.manual_seed(42)

In [1]:
X = torch.randn(N, D_in)

W_true = torch.tensor([[2.0]], requires_grad=True)
b_true = torch.tensor(1.0, requires_grad=True)
print(f"X.shape: {X.shape}")
print(f"W_true: {W_true}")
print(f"b_true: {b_true}")

NameError: name 'torch' is not defined